In [1]:
import pandas as pd
import audioread
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import recall_score, precision_score, classification_report

%matplotlib inline

In [3]:
# считываем данные 
music = pd.read_csv("Desktop/BI/BI_2020-2021_Python/Music_classification/out_40_corrected.csv", encoding ='mac_cyrillic', sep=";")
music

,student,song,coarse_genre,genre1,genre2,genre3,zero_crossing_rate,spectral_centroid,spectral_rolloff,mfcc,...,chroma_frequencies_3,chroma_frequencies_4,chroma_frequencies_5,chroma_frequencies_6,chroma_frequencies_7,chroma_frequencies_8,chroma_frequencies_9,chroma_frequencies_10,chroma_frequencies_11,chroma_frequencies_12
0,Евдокимова_Анастасия,Passacalia-Hendel,classic,instrumental,NaN,NaN,0.045798,748.314035,1269.271721,-11.941515,...,0.150021,0.153033,0.167364,0.191547,0.205490,0.187953,0.168512,0.154818,0.146623,0.138912
1,Журбенко Петр,Johannes_Passion—Johann_Sebastian_Bach,classic,aria,NaN,NaN,0.076417,1414.632434,2666.842841,-10.983263,...,0.319919,0.327633,0.360628,0.388893,0.381388,0.385996,0.293817,0.420031,0.248379,0.282112
2,Иванова Евгения,Канон_Для_Струнного_Оркестра-Иоганн_Пахельбель,classic,NaN,NaN,NaN,0.085291,1490.988501,2839.114715,-5.051047,...,0.216023,0.297068,0.252765,0.303570,0.315133,0.249486,0.221325,0.201776,0.195795,0.202072
3,Людмила_Проценко,A_Simple_Life-Brian_Crain,classic,neo-classical,new_age_piano,NaN,0.042886,656.653479,1070.636358,-2.400885,...,0.347152,0.267500,0.206898,0.222848,0.253309,0.282360,0.310962,0.263674,0.224395,0.235438
4,Никанорова Даша,Adagio_in_G_Minor-Albinoni,classic,instrumental,NaN,NaN,0.102339,1768.348182,3328.242111,-7.314991,...,0.392298,0.412282,0.485981,0.478761,0.525003,0.381633,0.471162,0.381619,0.338369,0.341242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,Шемякина Аня,The_Struts_-_Primadonna_Like_Me,rock,alternative_rock,glam_rock,NaN,0.153660,3041.577527,6378.389715,9.809298,...,0.503473,0.499696,0.385848,0.354051,0.368450,0.351270,0.441679,0.446410,0.542506,0.320269
320,Aleksei Zverev,Попрыгун_и_Гвозди-На_Убере,rock,indie,alternative_pop,NaN,0.086736,2103.759219,4411.500950,3.254812,...,0.321782,0.289839,0.286502,0.307791,0.315518,0.292220,0.267222,0.281542,0.275587,0.270769
321,Aleksei Zverev,Попрыгун_и_Гвозди-Рыбки,rock,indie,alternative_pop,NaN,0.084793,2049.282500,4381.956846,3.578090,...,0.460827,0.466181,0.423808,0.267113,0.187591,0.210219,0.308529,0.347520,0.287365,0.459892
322,Aleksei Zverev,Попрыгун_и_Гвозди-Черёмуха,rock,indie,alternative_pop,NaN,0.071818,2010.463465,4376.270481,5.899025,...,0.355812,0.447199,0.318935,0.252392,0.371693,0.407885,0.296010,0.305457,0.496221,0.452474


In [4]:
# Убираем фичи с NaN (random forest не умеет с ними работать)
music_clean = music.drop(["genre1","genre2", "genre3"], axis=1)
music_clean["coarse_genre"].value_counts()

electronic      79
rock            75
metal           53
pop             42
indie           27
hip-hop         26
instrumental     9
classic          8
jazz             5
Name: coarse_genre, dtype: int64

In [5]:
# оставляем только наиболее многочисленные группы
music_clean_2 = music_clean.loc[music['coarse_genre'].isin(["rock","metal", "electronic", "pop", "indie", "hip-hop"])]

In [6]:
# разделяем фичи и предсказываемую переменную и оцениваем корреляцию переменных
X = music_clean_2.drop(["student", "song", "coarse_genre"], axis=1)
y = music_clean_2.loc[:,"coarse_genre"]
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm')

,zero_crossing_rate,spectral_centroid,spectral_rolloff,mfcc,tonal_centroid,beats_per_minute,spectral_bandwidth,chroma_frequencies_1,chroma_frequencies_2,chroma_frequencies_3,chroma_frequencies_4,chroma_frequencies_5,chroma_frequencies_6,chroma_frequencies_7,chroma_frequencies_8,chroma_frequencies_9,chroma_frequencies_10,chroma_frequencies_11,chroma_frequencies_12
zero_crossing_rate,1.000000,0.881510,0.788890,0.173998,-0.088495,0.161113,0.636694,0.029455,0.031905,0.086386,0.135856,0.080488,0.048813,0.055341,0.116007,0.198099,0.132791,0.131802,0.123163
spectral_centroid,0.881510,1.000000,0.978312,0.334976,-0.146682,0.152893,0.900452,0.119837,0.164391,0.202520,0.233590,0.163044,0.149794,0.181979,0.230215,0.280108,0.220893,0.219135,0.241548
spectral_rolloff,0.788890,0.978312,1.000000,0.368582,-0.159888,0.139955,0.954025,0.137222,0.188598,0.224353,0.249344,0.176642,0.176725,0.209887,0.252846,0.282988,0.229793,0.224247,0.251938
mfcc,0.173998,0.334976,0.368582,1.000000,-0.100620,-0.083878,0.376486,0.292158,0.324027,0.358819,0.366567,0.349438,0.345816,0.383663,0.382528,0.361134,0.397406,0.403738,0.302060
tonal_centroid,-0.088495,-0.146682,-0.159888,-0.100620,1.000000,0.028942,-0.173801,-0.044903,-0.026701,-0.016370,-0.101106,-0.070354,-0.102500,-0.096022,-0.121446,-0.108263,-0.089770,-0.094057,-0.125993
beats_per_minute,0.161113,0.152893,0.139955,-0.083878,0.028942,1.000000,0.103806,-0.022507,-0.008489,-0.034800,-0.014820,-0.002027,0.000492,-0.004409,0.053062,0.040451,0.050026,0.068865,0.049807
spectral_bandwidth,0.636694,0.900452,0.954025,0.376486,-0.173801,0.103806,1.000000,0.147639,0.198745,0.226707,0.233372,0.156732,0.172252,0.209088,0.238115,0.253946,0.217052,0.212783,0.252541
chroma_frequencies_1,0.029455,0.119837,0.137222,0.292158,-0.044903,-0.022507,0.147639,1.000000,0.798267,0.658356,0.514136,0.458784,0.438835,0.409418,0.343182,0.345334,0.377933,0.342267,0.322835
chroma_frequencies_2,0.031905,0.164391,0.188598,0.324027,-0.026701,-0.008489,0.198745,0.798267,1.000000,0.824515,0.620159,0.554231,0.531059,0.528004,0.449446,0.439926,0.436711,0.415696,0.426471
chroma_frequencies_3,0.086386,0.202520,0.224353,0.358819,-0.016370,-0.034800,0.226707,0.658356,0.824515,1.000000,0.804792,0.671391,0.568049,0.529668,0.461656,0.458291,0.443616,0.431220,0.460381


In [12]:
# получаем выборки для обучения и тестирования
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
# создаём классификатор
clf = RandomForestClassifier(criterion = "entropy")

In [14]:
# определяем параметры для настройки классификатора
parameters = {"n_estimators": range(1,100), 
              "min_samples_leaf": [4, 8, 12], 
             "max_depth": [3, 5, 7]}

In [24]:
# подстраиваем классификатор для максимальной точности
grid = GridSearchCV(clf, parameters, cv = 5)
grid.fit(X_train, y_train)

KeyboardInterrupt: 

In [20]:
# Оценка классификатора
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  electronic       0.62      0.72      0.67        25
     hip-hop       0.00      0.00      0.00         9
       indie       0.00      0.00      0.00         7
       metal       0.68      0.87      0.76        15
         pop       0.08      0.10      0.09        10
        rock       0.44      0.48      0.46        25

    accuracy                           0.48        91
   macro avg       0.30      0.36      0.33        91
weighted avg       0.41      0.48      0.45        91



In [21]:
# убираем автокоррелирующие фичи
X = music_clean_2.drop(["student", "song", "coarse_genre", "spectral_centroid", "spectral_rolloff", 
                       "chroma_frequencies_2", "chroma_frequencies_4", "chroma_frequencies_6", "chroma_frequencies_8",
                       "chroma_frequencies_10", "chroma_frequencies_12"], axis=1)
y = music_clean_2.loc[:,"coarse_genre"]
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm')

,zero_crossing_rate,mfcc,tonal_centroid,beats_per_minute,spectral_bandwidth,chroma_frequencies_1,chroma_frequencies_3,chroma_frequencies_5,chroma_frequencies_7,chroma_frequencies_9,chroma_frequencies_11
zero_crossing_rate,1.000000,0.173998,-0.088495,0.161113,0.636694,0.029455,0.086386,0.080488,0.055341,0.198099,0.131802
mfcc,0.173998,1.000000,-0.100620,-0.083878,0.376486,0.292158,0.358819,0.349438,0.383663,0.361134,0.403738
tonal_centroid,-0.088495,-0.100620,1.000000,0.028942,-0.173801,-0.044903,-0.016370,-0.070354,-0.096022,-0.108263,-0.094057
beats_per_minute,0.161113,-0.083878,0.028942,1.000000,0.103806,-0.022507,-0.034800,-0.002027,-0.004409,0.040451,0.068865
spectral_bandwidth,0.636694,0.376486,-0.173801,0.103806,1.000000,0.147639,0.226707,0.156732,0.209088,0.253946,0.212783
chroma_frequencies_1,0.029455,0.292158,-0.044903,-0.022507,0.147639,1.000000,0.658356,0.458784,0.409418,0.345334,0.342267
chroma_frequencies_3,0.086386,0.358819,-0.016370,-0.034800,0.226707,0.658356,1.000000,0.671391,0.529668,0.458291,0.431220
chroma_frequencies_5,0.080488,0.349438,-0.070354,-0.002027,0.156732,0.458784,0.671391,1.000000,0.629770,0.526853,0.461297
chroma_frequencies_7,0.055341,0.383663,-0.096022,-0.004409,0.209088,0.409418,0.529668,0.629770,1.000000,0.703646,0.521443
chroma_frequencies_9,0.198099,0.361134,-0.108263,0.040451,0.253946,0.345334,0.458291,0.526853,0.703646,1.000000,0.632302


In [22]:
# повторное формирование выборок
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
# повторное обучение классификатора 
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [31]:
# оценка качества предсказаний
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  electronic       0.72      0.72      0.72        25
     hip-hop       0.50      0.11      0.18         9
       indie       0.33      0.14      0.20         7
       metal       0.58      0.73      0.65        15
         pop       0.17      0.20      0.18        10
        rock       0.40      0.48      0.44        25

    accuracy                           0.49        91
   macro avg       0.45      0.40      0.39        91
weighted avg       0.50      0.49      0.48        91

